In [49]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import read_inputs, compute_cycle_efficiency
# import compute_required_area
import test as compute_required_area
from th_functions import *

import scipy.optimize as spo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
def cost_calc(x, print_res=False):
    # Assume x to have the following structure
    Tloss_i, dT_i, d_channel, dP_p, dP_s = x
    inputs = read_inputs.read_inputs()
    inputs["Intermediate Tloss"] = Tloss_i
    inputs["Intermediate dTMultiplier"] = dT_i
    inputs["Channel Diameter (m)"] = d_channel
    inputs["Primary Pressure Drop (kPa)"] = dP_p
    inputs["Secondary Pressure Drop (kPa)"] = dP_s
    try:
        area_results = compute_required_area.compute_required_area(inputs)
        channel_thickness = 3*inputs["Plate thickness (m)"] + inputs["Channel Diameter (m)"]
        channel_width = inputs["Plate thickness (m)"] + inputs["Channel Diameter (m)"]
        channel_volume = channel_thickness*channel_width*area_results["Heat Exchanger Length (m)"]
        HX_volume = sum(channel_volume*area_results["Number of Channels"])
        inflation = 1.62
        cost_conversion = 132000          # $/m3
        cost = HX_volume*132000*inflation # $
    except:
        return 8e5
    if print_res:
        print("Vol:  ", channel_volume*area_results["Number of Channels"])
        print('Cost: ', cost)
        print(pd.Series(inputs))
        print(pd.Series(area_results))
        print(pd.Series(compute_cycle_efficiency.compute_cycle_efficiency(inputs)))
    return cost

bounds = spo.Bounds(
    lb=[5, 1.001, 5e-4, 10, 10],
    ub=[25, 1.3, 8e-3, 150, 150],
)
res = spo.minimize(cost_calc, [15.3, 1.127, 1.354e-3, 104.2, 200], bounds=bounds)
print(res)
cost_calc(res.x, print_res=True)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 397619.4634388773
        x: [ 1.626e+01  1.107e+00  1.345e-03  1.042e+02  1.500e+02]
      nit: 19
      jac: [-2.602e+00 -2.790e+01 -2.209e+03  0.000e+00 -4.083e+02]
     nfev: 216
     njev: 36
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
Vol:   [0.70017   1.1592551]
Cost:  397619.4634388773
Thermal Power (MW)                              30
Primary Fluid                               Sodium
Primary Hot Temperature (C)                    550
Primary Cold Temperature (C)                   400
Primary Mass Flow Rate (kg/s)                  NaN
Primary Pressure (kPa)                         300
Secondary Fluid                      CarbonDioxide
Secondary Hot Temperature (C)                  500
Secondary Cold Temperature (C)                 300
Secondary Mass Flow Rate (kg/s)                NaN
Secondary Pressure (kPa)                     25000
Intermediate Tloss               

397619.4634388773